## Notebook for spike sorting from .rhd data using the Pipeline of mountainsort (linux channel has pipeline)
Uses:
    - intan2kwik (https://github.com/zekearneodo/intan2kwik/blob/master/README.md)
    - mountainlab suite(https://github.com/flatironinstitute/mountainlab-js)
    - mountainsort https://github.com/flatironinstitute/mountainsort_examples/blob/master/README.md
    - mountainsort examples https://github.com/flatironinstitute/mountainsort_examples/blob/master/README.md

In [1]:
import os
import glob
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import subprocess
from importlib import reload

# mountainlab imports
from pipefinch.neural.convert import intan
from pipefinch.neural.sort.mountain import core as msc

import logging

# Setup the logger
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

ch = logging.StreamHandler()
ch.setLevel(logging.INFO)
ch.setFormatter(formatter)
logger.addHandler(ch)

logger.info('Logger set')

/home/zinch/conda/envs/msort/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/zinch/conda/envs/msort/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/zinch/conda/envs/msort/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 88 from C header, got 96 from PyObject
  return f(*args, **kwds)
2018-11-22 19:40:30,139 - root - INFO - Logger set


In [2]:
os.environ['OMP_NUM_THREADS'] = '8'
os.environ['MKL_NUM_THREADS'] = '8'
os.environ['NUMEXPR_NUM_THREADS'] = '8'

### Session parameters and raw files

In [3]:
raw_folder = os.path.abspath('/home/zinch/data/o8g8/neural/chronic_20181122_2500_01')

In [3]:
exp_base = os.path.abspath('/home/zinch/data/')

file_sys = {'exp_base': exp_base}

sess_par = {'bird': 'o8g8',
           'sess': 'chronic_20181122_2500_01',
           'rec': 0}

file_sys_defaults = {'exp_base': os.path.abspath('/data/experiment')}


def file_names(sess_par, file_sys=None):
    f_s = unroll_file_sys(file_sys_defaults, file_sys)    
    
    f_n = {}
    f_n['raw'] = os.path.join(f_s['raw'], bird)

In [4]:
exp_base

'/home/zinch/data'

In [5]:
raw_folder = os.path.join(exp_base, sess_par['bird'], 'neural', sess_par['sess'])
logger.info('Raw folder {}'.format(raw_folder))
glob.glob(os.path.join(raw_folder, '*.rhd'))

2018-11-22 20:00:13,163 - root - INFO - Raw folder /home/zinch/data/o8g8/neural/chronic_20181122_2500_01


['/home/zinch/data/o8g8/neural/chronic_20181122_2500_01/raw_intan_181122_114745.rhd',
 '/home/zinch/data/o8g8/neural/chronic_20181122_2500_01/raw_intan_181122_105245.rhd',
 '/home/zinch/data/o8g8/neural/chronic_20181122_2500_01/raw_intan_181122_112745.rhd',
 '/home/zinch/data/o8g8/neural/chronic_20181122_2500_01/raw_intan_181122_105745.rhd',
 '/home/zinch/data/o8g8/neural/chronic_20181122_2500_01/raw_intan_181122_102744.rhd',
 '/home/zinch/data/o8g8/neural/chronic_20181122_2500_01/raw_intan_181122_114245.rhd',
 '/home/zinch/data/o8g8/neural/chronic_20181122_2500_01/raw_intan_181122_103745.rhd',
 '/home/zinch/data/o8g8/neural/chronic_20181122_2500_01/raw_intan_181122_110745.rhd',
 '/home/zinch/data/o8g8/neural/chronic_20181122_2500_01/raw_intan_181122_113745.rhd',
 '/home/zinch/data/o8g8/neural/chronic_20181122_2500_01/raw_intan_181122_110245.rhd',
 '/home/zinch/data/o8g8/neural/chronic_20181122_2500_01/raw_intan_181122_103245.rhd',
 '/home/zinch/data/o8g8/neural/chronic_20181122_2500_0

In [6]:
rhd_files = glob.glob(os.path.join(raw_folder, '*.rhd'))
rhd_files.sort()
pd_f = pd.DataFrame(rhd_files, columns=['f_path'])

pd_f['stamp'] = pd_f['f_path'].apply(lambda x: int(os.path.split(x)[-1].split('.')[0].split('_')[-1]))

# Select some of the files
sess_start = 0
sess_end = 235959
pd_session = pd_f.loc[(pd_f['stamp'] > sess_start) & (pd_f['stamp'] < sess_end), :]
pd_session.index.size

rhd_list = pd_session['f_path'].values.tolist()[:]

## Scripts for sorting

### prep the files with their nice formats, locations and names


In [7]:
rhd_list

['/home/zinch/data/o8g8/neural/chronic_20181122_2500_01/raw_intan_181122_102744.rhd',
 '/home/zinch/data/o8g8/neural/chronic_20181122_2500_01/raw_intan_181122_103245.rhd',
 '/home/zinch/data/o8g8/neural/chronic_20181122_2500_01/raw_intan_181122_103745.rhd',
 '/home/zinch/data/o8g8/neural/chronic_20181122_2500_01/raw_intan_181122_104245.rhd',
 '/home/zinch/data/o8g8/neural/chronic_20181122_2500_01/raw_intan_181122_104745.rhd',
 '/home/zinch/data/o8g8/neural/chronic_20181122_2500_01/raw_intan_181122_105245.rhd',
 '/home/zinch/data/o8g8/neural/chronic_20181122_2500_01/raw_intan_181122_105745.rhd',
 '/home/zinch/data/o8g8/neural/chronic_20181122_2500_01/raw_intan_181122_110245.rhd',
 '/home/zinch/data/o8g8/neural/chronic_20181122_2500_01/raw_intan_181122_110745.rhd',
 '/home/zinch/data/o8g8/neural/chronic_20181122_2500_01/raw_intan_181122_111245.rhd',
 '/home/zinch/data/o8g8/neural/chronic_20181122_2500_01/raw_intan_181122_111745.rhd',
 '/home/zinch/data/o8g8/neural/chronic_20181122_2500_0

In [8]:
ss_folder = os.path.join(exp_base, sess_par['bird'], 'ss_data', sess_par['sess'])

file_paths, out_folder = msc.make_paths(ss_folder)
os.makedirs(ss_folder, exist_ok=True)
#todo:
# meta and pars from intan files et al
s_f = 20000
ds_params = {'samplerate': s_f}

with open(file_paths['params'], 'w') as fp:
    json.dump(ds_params, fp)
    logger.info('Created session par files {}'.format(file_paths['params']))
    
# create the mda file from the rhd_list
intan_meta, samples_in = intan.intan_to_bin(rhd_list[:], file_paths['mda'], variant='mda')


2018-11-22 20:01:11,694 - root - INFO - Created session par files /home/zinch/data/o8g8/ss_data/chronic_20181122_2500_01/params.json
2018-11-22 20:01:11,696 - pipefinch.neural.convert - INFO - Creating the output file /home/zinch/data/o8g8/ss_data/chronic_20181122_2500_01/raw.mda
2018-11-22 20:01:11,697 - pipefinch.neural.convert - INFO - File 0/17
file raw_intan_181122_102744.rhd: 100%|██████████| 100008/100008 [00:09<00:00, 11014.43it/s]
2018-11-22 20:01:21,603 - pipefinch.neural.convert - INFO - File 1/17
file raw_intan_181122_103245.rhd: 100%|██████████| 100008/100008 [00:08<00:00, 11909.43it/s]
2018-11-22 20:01:30,872 - pipefinch.neural.convert - INFO - File 2/17
file raw_intan_181122_103745.rhd: 100%|██████████| 100008/100008 [00:08<00:00, 11876.83it/s]
2018-11-22 20:01:40,129 - pipefinch.neural.convert - INFO - File 3/17
file raw_intan_181122_104245.rhd: 100%|██████████| 100008/100008 [00:08<00:00, 11959.77it/s]
2018-11-22 20:01:49,342 - pipefinch.neural.convert - INFO - File 4/

In [9]:
ss_folder

'/home/zinch/data/o8g8/ss_data/chronic_20181122_2500_01'

In [10]:
file_paths

{'mda': '/home/zinch/data/o8g8/ss_data/chronic_20181122_2500_01/raw.mda',
 'params': '/home/zinch/data/o8g8/ss_data/chronic_20181122_2500_01/params.json',
 'geom': '/home/zinch/data/o8g8/ss_data/chronic_20181122_2500_01/geom.csv',
 'filt': '/home/zinch/data/o8g8/ss_data/chronic_20181122_2500_01/sort_out/filt.mda.prv',
 'pre': '/home/zinch/data/o8g8/ss_data/chronic_20181122_2500_01/sort_out/pre.mda.prv',
 'firings': '/home/zinch/data/o8g8/ss_data/chronic_20181122_2500_01/sort_out/firings.mda',
 'firings_curated': '/home/zinch/data/o8g8/ss_data/chronic_20181122_2500_01/sort_out/firings_curated.mda',
 'cluster_metrics': '/home/zinch/data/o8g8/ss_data/chronic_20181122_2500_01/sort_out/cluster_metrics.json'}

In [11]:
msc.read_dataset_params(ss_folder)

{'samplerate': 20000}

In [18]:
reload(msc)

<module 'pipefinch.neural.sort.mountain.core' from '/home/ezequiel/repos/pipefinch/pipefinch/neural/sort/mountain/core.py'>

In [12]:
from mountainlab_pytools import mlproc as mlp
Pipeline=mlp.initPipeline()

JSProxyWidget(status='Not yet rendered')

In [15]:
output_dir = os.path.join(ss_folder, 'sort_out')
# Pipeline was failing, need to debug this ('NameError: name 'widgets' is not defined')
# with Pipeline:
#     msc.sort_dataset(file_paths=file_paths, adjacency_radius=-1, detect_threshold=3, dispatch_method='add')

msc.sort_dataset(file_paths=file_paths, adjacency_radius=-1, detect_threshold=3, dispatch_method='run')

2018-11-22 20:07:45,923 - pipefinch.sort.mountain.comre - INFO - Bandpass filter


RUNNING: ml-run-process ephys.bandpass_filter --inputs timeseries:/home/zinch/data/o8g8/ss_data/chronic_20181122_2500_01/raw.mda --parameters freq_max:6000 freq_min:300 samplerate:20000 --outputs timeseries_out:/home/zinch/data/o8g8/ss_data/chronic_20181122_2500_01/sort_out/filt.mda.prv


2018-11-22 20:09:00,226 - pipefinch.sort.mountain.comre - INFO - Whitening


[ Getting processor spec... ]
[ Checking inputs and substituting prvs ... ]
[ Computing process signature ... ]
Process signature: cf3df595f9f4ec9820427dc6e37ca597c803222b
[ Checking outputs... ]
{"timeseries_out":"/home/zinch/data/o8g8/ss_data/chronic_20181122_2500_01/sort_out/filt.mda.prv"}
Processing ouput - /home/zinch/data/o8g8/ss_data/chronic_20181122_2500_01/sort_out/filt.mda.prv
false
{"timeseries_out":"/tmp/mountainlab-tmp/output_cf3df595f9f4ec9820427dc6e37ca597c803222b_timeseries_out.mda"}
[ Checking process cache ... ]
[ Creating temporary directory ... ]
[ Creating links to input files... ]
[ Preparing temporary outputs... ]
Processing ouput - /tmp/mountainlab-tmp/output_cf3df595f9f4ec9820427dc6e37ca597c803222b_timeseries_out.mda
false
[ Initializing process ... ]
[ Running ... ] /home/zinch/conda/envs/msort/bin/python3 /home/zinch/conda/envs/msort/etc/mountainlab/packages/ml_ephys/preprocessing/preprocessing.py.mp ephys.bandpass_filter --_tempdir=/tmp/mountainlab-tmp/tempd

2018-11-22 20:09:42,216 - pipefinch.sort.mountain.comre - INFO - Sorting


...

RUNNING: ml-run-process ms4alg.sort --inputs timeseries:/home/zinch/data/o8g8/ss_data/chronic_20181122_2500_01/sort_out/pre.mda.prv --parameters adjacency_radius:-1 detect_sign:1 detect_threshold:3 --outputs firings_out:/home/zinch/data/o8g8/ss_data/chronic_20181122_2500_01/sort_out/firings.mda
[ Getting processor spec... ]
/home/zinch/conda/envs/msort/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 88 from C header, got 96 from PyObject
[ Checking inputs and substituting prvs ... ]
return f(*args, **kwds)
[ Computing process signature ... ]



2018-11-22 20:21:20,413 - pipefinch.sort.mountain.comre - INFO - Getting cluster metrics


Process signature: e30381f4405849b8515bf0465c1f1058f05913b1
[ Checking outputs... ]
{"firings_out":"/home/zinch/data/o8g8/ss_data/chronic_20181122_2500_01/sort_out/firings.mda"}
Processing ouput - /home/zinch/data/o8g8/ss_data/chronic_20181122_2500_01/sort_out/firings.mda
false
{"firings_out":"/home/zinch/data/o8g8/ss_data/chronic_20181122_2500_01/sort_out/firings.mda"}
[ Checking process cache ... ]
[ Creating temporary directory ... ]
[ Creating links to input files... ]
[ Preparing temporary outputs... ]
Processing ouput - /home/zinch/data/o8g8/ss_data/chronic_20181122_2500_01/sort_out/firings.mda
false
[ Initializing process ... ]
[ Running ... ] /home/zinch/conda/envs/msort/bin/python3 /home/zinch/conda/envs/msort/etc/mountainlab/packages/ml_ms4alg/ms4alg_spec.py.mp ms4alg.sort --_tempdir=/tmp/mountainlab-tmp/tempdir_e30381f440_E5wmB2 --timeseries=/tmp/mountainlab-tmp/tempdir_e30381f440_E5wmB2/input_timeseries_2U9iPqgD.mda --geom= --firings_out=/tmp/mountainlab-tmp/tempdir_e30381f

2018-11-22 20:22:20,957 - pipefinch.sort.mountain.comre - INFO - Automatically curating


[ Getting processor spec... ]
[ Checking inputs and substituting prvs ... ]
[ Computing process signature ... ]
Process signature: e553f43cbb62e4a3b17d454e2fa6113bf9d41a71
[ Checking outputs... ]
{"metrics_out":"/home/zinch/data/o8g8/ss_data/chronic_20181122_2500_01/sort_out/cluster_metrics.json"}
Processing ouput - /home/zinch/data/o8g8/ss_data/chronic_20181122_2500_01/sort_out/cluster_metrics.json
false
{"metrics_out":"/home/zinch/data/o8g8/ss_data/chronic_20181122_2500_01/sort_out/cluster_metrics.json"}
[ Checking process cache ... ]
[ Creating temporary directory ... ]
[ Creating links to input files... ]
[ Preparing temporary outputs... ]
Processing ouput - /home/zinch/data/o8g8/ss_data/chronic_20181122_2500_01/sort_out/cluster_metrics.json
false
[ Initializing process ... ]
[ Running ... ] /home/zinch/conda/envs/msort/etc/mountainlab/packages/ms3.mp ms3.combine_cluster_metrics --_tempdir=/tmp/mountainlab-tmp/tempdir_e553f43cbb_1khda5 --metrics_list=/tmp/mountainlab-tmp/tempdir_e5

In [14]:
msc.sort_dataset?

Signature: msc.sort_dataset(*, file_paths:dict, freq_min:int=300, freq_max:int=6000, adjacency_radius:int=1, detect_threshold:float=3, dispatch_method:str='run', opts:dict={})
Docstring: <no docstring>
File:      ~/repos/pipefinch/pipefinch/neural/sort/mountain/core.py
Type:      function


In [33]:
file_paths

{'mda': '/Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/raw.mda',
 'params': '/Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/params.json',
 'geom': '/Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/geom.csv',
 'filt': '/Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/sort_out/filt.mda.prv',
 'pre': '/Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/sort_out/pre.mda.prv',
 'firings': '/Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/sort_out/firings.mda',
 'firings_curated': '/Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/sort_out/firings_curated.mda',
 'cluster_metrics': '/Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/sort_out/cluster_metrics.json'}

In [10]:
cmd = 'env'
p = subprocess.Popen([cmd], stdout=subprocess.PIPE)
ans = p.communicate()
print(ans)

(b'MANPATH=/opt/local/share/man/:\nTERM_PROGRAM=Apple_Terminal\nSHELL=/bin/bash\nTERM=xterm-color\nTMPDIR=/var/folders/gk/7z94s6yj40n4ggn5dl62xhbc0000gn/T/\nCONDA_SHLVL=1\nApple_PubSub_Socket_Render=/private/tmp/com.apple.launchd.rxM5aew32h/Render\nCONDA_PROMPT_MODIFIER=(mountain) \nTERM_PROGRAM_VERSION=404\nTERM_SESSION_ID=E6D6F9C0-AA53-473E-B5D3-B4EE02E408C3\nUSER=zeke\nCONDA_EXE=/Users/zeke/anaconda3/bin/conda\nML_CONDA_PACKAGES_DIR=/Users/zeke/anaconda3/envs/mountain/etc/mountainlab/packages\nSSH_AUTH_SOCK=/private/tmp/com.apple.launchd.ZGXuue9qT4/Listeners\nML_CONDA_DIR=/Users/zeke/anaconda3/envs/mountain/etc/mountainlab\nPATH=/Users/zeke/anaconda3/envs/mountain/bin:/Users/zeke/google-cloud-sdk/bin:/opt/local/bin:/opt/local/sbin:/Library/Frameworks/Python.framework/Versions/3.7/bin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin\nCONDA_PREFIX=/Users/zeke/anaconda3/envs/mountain\nPWD=/Users/zeke/repos\nML_CONFIG_DIRECTORY=/Users/zeke/anaconda3/envs/mountain/etc/mountainlab\nLANG=en_US

In [16]:
show_args_dict = {'raw': file_paths['mda'], 
             'filt': file_paths['filt'],
             'pre': file_paths['pre'],
            'samplereate': ds_params['samplerate'],
            'firings': file_paths['firings_curated'],
            'cluster_metrics': file_paths['cluster_metrics']}

cmd_args = ['--{} {}'.format(k, v) for k, v in show_args_dict.items()]
cmd = 'qt-mountainview'

logger.info('View command is {}'.format([cmd, ' '.join(cmd_args)]))
p = subprocess.Popen([cmd, ' '.join(cmd_args)], stdout=subprocess.PIPE)


2018-10-01 17:34:53,025 - root - INFO - View command is ['qt-mountainview', '--raw /data/experiment/ss_data/o3g3/awake_20180908_2925_01/raw.mda --filt /data/experiment/ss_data/o3g3/awake_20180908_2925_01/sort_out/filt.mda.prv --pre /data/experiment/ss_data/o3g3/awake_20180908_2925_01/sort_out/pre.mda.prv --samplereate 20000 --firings /data/experiment/ss_data/o3g3/awake_20180908_2925_01/sort_out/firings_curated.mda --cluster_metrics /data/experiment/ss_data/o3g3/awake_20180908_2925_01/sort_out/cluster_metrics.json']


In [17]:
print(' '.join(['qt-mountainview'] + cmd_args))

qt-mountainview --raw /data/experiment/ss_data/o3g3/awake_20180908_2925_01/raw.mda --filt /data/experiment/ss_data/o3g3/awake_20180908_2925_01/sort_out/filt.mda.prv --pre /data/experiment/ss_data/o3g3/awake_20180908_2925_01/sort_out/pre.mda.prv --samplereate 20000 --firings /data/experiment/ss_data/o3g3/awake_20180908_2925_01/sort_out/firings_curated.mda --cluster_metrics /data/experiment/ss_data/o3g3/awake_20180908_2925_01/sort_out/cluster_metrics.json


# other viewing examples

ev-templates output/templates.mda.prv
ev-timeseries dataset/raw.mda.prv --firings output/firings.mda.prv --samplerate=30000
qt-mountainview --raw dataset/raw.mda.prv --filt output/filt.mda.prv --pre output/pre.mda.prv --samplerate 30000 --firings output/firings.mda.prv
